In [2]:
import tensorflow as tf
import numpy as np
import faiss
import pandas as pd 

In [3]:
songs = pd.read_csv('input/songs.csv')
song_extra_info = pd.read_csv('input/song_extra_info.csv')

In [15]:
train_df = pd.read_csv('preprocess/train.csv')
val_df = pd.read_csv('preprocess/val.csv')

In [17]:
# msno, song_id 编码
from sklearn.preprocessing import LabelEncoder
def encode_label(train_df, val_df, field):
    le = LabelEncoder()
    le.fit(list(train_df[field].unique()) + list(val_df[field].unique()))
    train_df[field] = le.transform(train_df[field])
    val_df[field] = le.transform(val_df[field])
    return train_df, val_df

train_df, val_df  = encode_label(train_df, val_df, 'msno')
train_df, val_df = encode_label(train_df, val_df, 'song_id')

In [18]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
label, user_idx, song_idx = train_df['target'].values, train_df['msno'].values, train_df['song_id'].values

song_user_matrix = csr_matrix((label, (song_idx, user_idx)))

dimension = 16
svd = TruncatedSVD(n_components=dimension)
song_vec = svd.fit_transform(song_user_matrix)

svd_columns = ['song_id'] + ['song_svd_' + str(c) for c in range(dimension)]
song_vec_pretrain = pd.DataFrame(np.concatenate([np.reshape(range(len(song_vec)), [-1, 1]), song_vec], axis=1), 
                                 columns=svd_columns)